Resources:
https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/TAPAS/Fine_tuning_TapasForQuestionAnswering_on_SQA.ipynb#scrollTo=QP4AHMxFujhV


In [37]:
#tokenizer = TapasTokenizer.from_pretrained('google/tapas-base-finetuned-wtq')
#tokenizer = TapasTokenizer.from_pretrained('google/tapas-large-finetuned-wtq')
model = TapasForQuestionAnswering.from_pretrained('google/tapas-base-finetuned-wtq')
#model = TapasForQuestionAnswering.from_pretrained('google/tapas-large-finetuned-wtq')

from transformers import AutoTokenizer, AutoModelForTableQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("google/tapas-large-finetuned-sqa")
#model = AutoModelForTableQuestionAnswering.from_pretrained("google/tapas-large-finetuned-sqa")

In [38]:
import pandas as pd
import datetime

In [39]:
#from transformers import AutoTokenizer, AutoModelForTableQuestionAnswering
import torch

#model = TapasForQuestionAnswering.from_pretrained("google/tapas-base")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

TapasForQuestionAnswering(
  (tapas): TapasModel(
    (embeddings): TapasEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(1024, 768)
      (token_type_embeddings_0): Embedding(3, 768)
      (token_type_embeddings_1): Embedding(256, 768)
      (token_type_embeddings_2): Embedding(256, 768)
      (token_type_embeddings_3): Embedding(2, 768)
      (token_type_embeddings_4): Embedding(256, 768)
      (token_type_embeddings_5): Embedding(256, 768)
      (token_type_embeddings_6): Embedding(10, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): TapasEncoder(
      (layer): ModuleList(
        (0): TapasLayer(
          (attention): TapasAttention(
            (self): TapasSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
 

In [40]:
# set 10 day lookback window
periodend = datetime.datetime.now().isoformat()
periodstart = (datetime.datetime.now()- datetime.timedelta(days=10)).isoformat()
#Get count of incidents to determine limit to retrieve
num=pd.read_json("https://data.sfgov.org/resource/RowID.json?$select=COUNT(incident_number)&$where=call_date%20between%20%27"
                 +str(periodstart)+"%27%20AND%20%27"+str(periodend)+"%27")
maxNumber=num.iloc[0,0]
print('The number of incidents in the set is',maxNumber)
# Use record to write query call to API to get all incidents needed/bypass API default of 1000
query_str=("https://data.sfgov.org/resource/RowID.json?$where=call_date%20between%20%27"
           +str(periodstart)+"%27%20AND%20%27"
           +str(periodend)+"%27&$limit="+str(maxNumber))
cfs_data=pd.read_json(query_str)
print(cfs_data.shape)
cfs_data.sample(5)

The number of incidents in the set is 7562
(7562, 34)


,call_number,unit_id,incident_number,call_type,call_date,watch_date,received_dttm,entry_dttm,dispatch_dttm,response_dttm,...,number_of_alarms,unit_type,unit_sequence_in_call_dispatch,fire_prevention_district,supervisor_district,neighborhoods_analysis_boundaries,rowid,case_location,transport_dttm,hospital_dttm
27,220231506,T13,22010921,Alarms,2022-01-23T00:00:00.000,2022-01-23T00:00:00.000,2022-01-23T13:27:45.000,2022-01-23T13:29:10.000,2022-01-23T13:29:17.000,2022-01-23T13:30:29.000,...,1,TRUCK,2,1,3,North Beach,220231506-T13,"{'type': 'Point', 'coordinates': [-122.4040784...",NaN,NaN
7107,220311369,E16,22014547,Medical Incident,2022-01-31T00:00:00.000,2022-01-31T00:00:00.000,2022-01-31T12:09:07.000,2022-01-31T12:11:35.000,2022-01-31T12:11:49.000,2022-01-31T12:13:03.000,...,1,ENGINE,3,4,2,Marina,220311369-E16,"{'type': 'Point', 'coordinates': [-122.4329351...",NaN,NaN
975,220242946,E07,22011559,Alarms,2022-01-24T00:00:00.000,2022-01-24T00:00:00.000,2022-01-24T19:35:47.000,2022-01-24T19:37:46.000,2022-01-24T19:38:33.000,2022-01-24T19:39:21.000,...,1,ENGINE,1,6,9,Mission,220242946-E07,"{'type': 'Point', 'coordinates': [-122.4193516...",NaN,NaN
5436,220292143,55,22013753,Medical Incident,2022-01-29T00:00:00.000,2022-01-29T00:00:00.000,2022-01-29T16:39:13.000,2022-01-29T16:40:14.000,2022-01-29T16:40:56.000,2022-01-29T16:41:33.000,...,1,MEDIC,2,10,10,Bayview Hunters Point,220292143-55,"{'type': 'Point', 'coordinates': [-122.3928969...",2022-01-29T17:14:47.000,2022-01-29T18:00:16.000
1980,220252397,SCRT4,22011936,Medical Incident,2022-01-25T00:00:00.000,2022-01-25T00:00:00.000,2022-01-25T16:43:05.000,2022-01-25T16:45:47.000,2022-01-25T16:59:59.000,2022-01-25T16:59:59.000,...,1,SUPPORT,1,6,8,Castro/Upper Market,220252397-SCRT4,"{'type': 'Point', 'coordinates': [-122.4282023...",NaN,NaN


In [41]:
# set 10 day lookback window
periodend = datetime.datetime.now().isoformat()
periodstart = (datetime.datetime.now()- datetime.timedelta(days=10)).isoformat()
#Get count of incidents to determine limit to retrieve
num=pd.read_json("https://data.sfgov.org/resource/RowID.json?$select=COUNT(incident_number)&$where=call_date%20between%20%27"
                 +str(periodstart)+"%27%20AND%20%27"+str(periodend)+"%27")
maxNumber=num.iloc[0,0]
print('The number of incidents in the set is',maxNumber)
# Use record to write query call to API to get all incidents needed/bypass API default of 1000
query_str=("https://data.sfgov.org/resource/RowID.json?$where=call_date%20between%20%27"
           +str(periodstart)+"%27%20AND%20%27"
           +str(periodend)+"%27&$limit="+str(maxNumber))
cfs_data=pd.read_json(query_str)
cfs_data=cfs_data.dropna()
cfs_data= cfs_data.astype(str)
cfs_small=cfs_data.head(10)
cfs_data_tapas=cfs_small[['call_type','neighborhoods_analysis_boundaries']]
#cfs_data_tapas=cfs_small[['unit_type','call_type','neighborhoods_analysis_boundaries']]
cfs_data_tapas=cfs_data_tapas.rename(columns={"call_type":"incident","neighborhoods_analysis_boundaries" : "neighbourhood" })
cfs_data_tapas

In [42]:
#cfs_data_tapas=cfs_data_tapas.rename(columns={"unit_type":"unit","call_type":"incident","neighborhoods_analysis_boundaries" : "neighbourhood" })
cfs_data_tapas=cfs_data_tapas.rename(columns={"call_type":"incident","neighborhoods_analysis_boundaries" : "neighbourhood" })
cfs_data_tapas

,incident,neighbourhood
1,Medical Incident,Tenderloin
13,Medical Incident,Tenderloin
15,Medical Incident,Outer Richmond
19,Medical Incident,Bayview Hunters Point
20,Medical Incident,South of Market
41,Medical Incident,Chinatown
51,Medical Incident,West of Twin Peaks
55,Medical Incident,Marina
56,Medical Incident,Chinatown
57,Medical Incident,Bayview Hunters Point


In [43]:
queries = ["What unit types are there?", "How many incidents are in Tenderloin?"]

inputs = tokenizer(table=cfs_data_tapas, queries=queries, padding='max_length', truncation=True, return_tensors="pt")

logits = compute_prediction_sequence(model, inputs, device)

predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits.cpu().detach())

# handy helper function in case inference on Pandas dataframe
answers = []
for coordinates in predicted_answer_coordinates:
  if len(coordinates) == 1:
    # only a single cell:
    answers.append(cfs_data_tapas.iat[coordinates[0]])
  else:
    # multiple cells
    cell_values = []
    for coordinate in coordinates:
      cell_values.append(cfs_data_tapas.iat[coordinate])
    answers.append(", ".join(cell_values))

display(cfs_data_tapas)
print("")
for query, answer in zip(queries, answers):
  print(query)
  print("Predicted answer: " + answer)

IndexError: iloc cannot enlarge its target object

In [44]:
encoding = tokenizer(table=table, queries=item.question, answer_coordinates=item.answer_coordinates, answer_text=item.answer_text,
                     truncation=True, padding="max_length", return_tensors="pt")
encoding.keys()

NameError: name 'table' is not defined

In [45]:
data = {'Actors': ["Brad Pitt", "Leonardo Di Caprio", "George Clooney"], 
        'Age': ["56", "45", "59"],
        'Number of movies': ["87", "53", "69"],
        'Date of birth': ["7 february 1967", "10 june 1996", "28 november 1967"]}
queries = ["How many movies has George Clooney played in?", "How old is he?", "What's his date of birth?"]

table = pd.DataFrame.from_dict(data)

inputs = tokenizer(table=table, queries=queries, padding='max_length', return_tensors="pt")
logits = compute_prediction_sequence(model, inputs, device)

NameError: name 'compute_prediction_sequence' is not defined

In [ ]:
predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits.cpu().detach())

In [ ]:
# handy helper function in case inference on Pandas dataframe
answers = []
for coordinates in predicted_answer_coordinates:
  if len(coordinates) == 1:
    # only a single cell:
    answers.append(table.iat[coordinates[0]])
  else:
    # multiple cells
    cell_values = []
    for coordinate in coordinates:
      cell_values.append(table.iat[coordinate])
    answers.append(", ".join(cell_values))

display(table)
print("")
for query, answer in zip(queries, answers):
  print(query)
  print("Predicted answer: " + answer)

In [46]:
cfs_data= cfs_data.astype(str)
cfs_small=cfs_data.head(500)
cfs_data_tapas=cfs_small[['call_date','call_type','neighborhoods_analysis_boundaries']]

queries = ["When was the most recent medical incident?", "How many incidents are in Tenderloin?"]

inputs = tokenizer(table=cfs_data_tapas, queries=queries, padding='max_length', return_tensors="pt",truncation=True)

logits = compute_prediction_sequence(model, inputs, device)

predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits.cpu().detach())

IndexError: iloc cannot enlarge its target object

In [ ]:
# handy helper function in case inference on Pandas dataframe
answers = []
for coordinates in predicted_answer_coordinates:
  if len(coordinates) == 1:
    # only a single cell:
    answers.append(cfs_data_tapas.iat[coordinates[0]])
  else:
    # multiple cells
    cell_values = []
    for coordinate in coordinates:
      cell_values.append(cfs_data_tapas.iat[coordinate])
    answers.append(", ".join(cell_values))

display(cfs_data_tapas)
print("")
for query, answer in zip(queries, answers):
  print(query)
  print("Predicted answer: " + answer)

In [47]:
cfs_data= cfs_data.astype(str)
cfs_small=cfs_data.head(500)
cfs_data_tapas=cfs_small[['call_date','call_type','neighborhoods_analysis_boundaries']]

#cfs_data_tapas=cfs_small[['call_number','call_date','call_type','unit_type','neighborhoods_analysis_boundaries']]
cfs_data_tapas=cfs_small[['call_date','call_type','neighborhoods_analysis_boundaries']]

cfs_data_tapas

,call_date,call_type,neighborhoods_analysis_boundaries
1,2022-01-23T00:00:00.000,Medical Incident,Tenderloin
13,2022-01-23T00:00:00.000,Medical Incident,Tenderloin
15,2022-01-23T00:00:00.000,Medical Incident,Outer Richmond
19,2022-01-23T00:00:00.000,Medical Incident,Bayview Hunters Point
20,2022-01-23T00:00:00.000,Medical Incident,South of Market
...,...,...,...
1869,2022-01-25T00:00:00.000,Medical Incident,Bayview Hunters Point
1871,2022-01-25T00:00:00.000,Medical Incident,South of Market
1881,2022-01-25T00:00:00.000,Medical Incident,Mission
1882,2022-01-25T00:00:00.000,Medical Incident,Sunset/Parkside


In [48]:
queries = ["How many medical incidents?","What type of calls are there?" ]

inputs = tokenizer(table=cfs_data_tapas, queries=queries, padding='max_length', return_tensors="pt",truncation=True)

IndexError: iloc cannot enlarge its target object

In [ ]:
logits = compute_prediction_sequence(model, inputs, device)

predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits.cpu().detach())

In [ ]:
# handy helper function in case inference on Pandas dataframe
answers = []
for coordinates in predicted_answer_coordinates:
  if len(coordinates) == 1:
    # only a single cell:
    answers.append(cfs_data_tapas.iat[coordinates[0]])
  else:
    # multiple cells
    cell_values = []
    for coordinate in coordinates:
      cell_values.append(cfs_data_tapas.iat[coordinate])
    answers.append(", ".join(cell_values))

display(cfs_data_tapas)
print("")
for query, answer in zip(queries, answers):
  print(query)
  print("Predicted answer: " + answer)

In [ ]:
from transformers import TapasConfig, TapasForQuestionAnswering
#config = TapasConfig('google-base-finetuned-wikisql-supervised')
#model = TapasForQuestionAnswering.from_pretrained('google/tapas-base', config=config)
model = TapasForQuestionAnswering.from_pretrained('google/tapas-base')

In [ ]:
logits = compute_prediction_sequence(model, inputs, device)
predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits.cpu().detach())

In [ ]:
# handy helper function in case inference on Pandas dataframe
answers = []
for coordinates in predicted_answer_coordinates:
  if len(coordinates) == 1:
    # only a single cell:
    answers.append(cfs_data_tapas.iat[coordinates[0]])
  else:
    # multiple cells
    cell_values = []
    for coordinate in coordinates:
      cell_values.append(cfs_data_tapas.iat[coordinate])
    answers.append(", ".join(cell_values))

display(cfs_data_tapas)
print("")
for query, answer in zip(queries, answers):
  print(query)
  print("Predicted answer: " + answer)

In [ ]:
config = TapasConfig.from_pretrained('google/tapas-base-finetuned-wtq')
model = TapasForQuestionAnswering.from_pretrained('google/tapas-base', config=config)
logits = compute_prediction_sequence(model, inputs, device)
predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits.cpu().detach())

# handy helper function in case inference on Pandas dataframe
answers = []
for coordinates in predicted_answer_coordinates:
  if len(coordinates) == 1:
    # only a single cell:
    answers.append(cfs_data_tapas.iat[coordinates[0]])
  else:
    # multiple cells
    cell_values = []
    for coordinate in coordinates:
      cell_values.append(cfs_data_tapas.iat[coordinate])
    answers.append(", ".join(cell_values))

display(cfs_data_tapas)
print("")
for query, answer in zip(queries, answers):
  print(query)
  print("Predicted answer: " + answer)

In [ ]:
# the base sized model with WikiSQL configuration
#config = TapasConfig('google-base-finetuned-wikisql-supervised')
from transformers import AutoTokenizer, AutoModelForTableQuestionAnswering
model = AutoModelForTableQuestionAnswering.from_pretrained("google/tapas-small-finetuned-wikisql-supervised")
logits = compute_prediction_sequence(model, inputs, device)
predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits.cpu().detach())

# handy helper function in case inference on Pandas dataframe
answers = []
for coordinates in predicted_answer_coordinates:
  if len(coordinates) == 1:
    # only a single cell:
    answers.append(cfs_data_tapas.iat[coordinates[0]])
  else:
    # multiple cells
    cell_values = []
    for coordinate in coordinates:
      cell_values.append(cfs_data_tapas.iat[coordinate])
    answers.append(", ".join(cell_values))

display(cfs_data_tapas)
print("")
for query, answer in zip(queries, answers):
  print(query)
  print("Predicted answer: " + answer)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/tapas-small-finetuned-wikisql-supervised")


In [ ]:
cfs_data= cfs_data.astype(str)
cfs_small=cfs_data.sample(100)
#cfs_data_tapas=cfs_small[['call_type','neighborhoods_analysis_boundaries']]
#cfs_data_tapas=cfs_data_tapas.rename(columns={"call_type":"incident type","neighborhoods_analysis_boundaries" : "neighbourhood" })
cfs_small

In [ ]:
list_of_list = [[]]
list_of_list[0] = list(cfs_small.columns)
list_of_list.extend(cfs_small.values.tolist())

In [ ]:
result = predict(list_of_list, ["How many medical incidents?","Which neighbour are there?"])

In [ ]:
queries = ["How many medical incidents?","Which neighbour are there?"]
inputs = tokenizer(table=cfs_data_tapas, queries=queries, padding='max_length', return_tensors="pt",truncation=True)

In [ ]:
model = AutoModelForTableQuestionAnswering.from_pretrained("google/tapas-small-finetuned-wikisql-supervised")
logits = compute_prediction_sequence(model, inputs, device)
predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits.cpu().detach())

# handy helper function in case inference on Pandas dataframe
answers = []
for coordinates in predicted_answer_coordinates:
  if len(coordinates) == 1:
    # only a single cell:
    answers.append(cfs_data_tapas.iat[coordinates[0]])
  else:
    # multiple cells
    cell_values = []
    for coordinate in coordinates:
      cell_values.append(cfs_data_tapas.iat[coordinate])
    answers.append(", ".join(cell_values))

display(cfs_data_tapas)
print("")
for query, answer in zip(queries, answers):
  print(query)
  print("Predicted answer: " + answer)